# Goal of script

Created on 07/03/2023 by Claire S

This script starts the clustering of data outputted by Script 2, which has then been manually cleaned

This script aims to find the best embedding to cluster the true positives and false positives

Things to pip install:

- spacy_sentence_bert (if using)
- huggingface
- sentence-transformers


Some sources:
- https://levelup.gitconnected.com/introduction-to-setfit-few-shot-text-classification-3fbf3a5b9b90
- https://www.youtube.com/watch?v=8h27lV8v8BU&ab_channel=HuggingFace

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# to import my own functions:
import pandas as pd
import sys
import ipynb
import re
import numpy as np
sys.path.append('/project/Xelix_Project/utils')

In [3]:
from ipynb.fs.full.Regex_html_Functions import clean_websites

In [4]:
# pandas options
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [5]:
# df = pd.read_pickle("data/Step0_Data/bank_change_emails_and_threads.pkl")
# df.to_csv('bank_change_emails.csv')

In [6]:
# # # import cleaned data
# df = pd.read_csv("data/checked_emails.csv")
# df = df.rename(columns={'bank-detail-location': 'bank_detail_loc'})
# df.head()

### Some details

In [7]:
# df['bank_detail_loc'].value_counts()

In [8]:
# df['email_type'].value_counts()

In [9]:
# df['category_slug'].value_counts()

In [10]:
# i = 1
# for row in df[0:10].itertuples():
#     print("#" * 70, "\n\n")
#     print(f'email index: {i}, email id: {row.email_id}')
#     i += 1
#     print(f'subject line:\t{row.subject}')
#     print(f'attachment name:\t{row.attachments}')
#     print(f'is part of thread:\t{bool(row.is_part_of_thread)}')
#     print(f'true positives:\t\t{row.true_positives}')
#     print(f'email type:\t\t{row.email_type}')
#     print(f'bank detail loc:\t{row.bank_detail_loc}')
#     print(f'{row.email_text_body}\n')

In [11]:
# select the columns we want:
# df.columns

In [12]:


# filter on the ones that are clear cases
#df_clear = df.query("uncertain == 0").reset_index(drop=True)

# # select 
# subset = df.loc[:, ['has_sort_code', 'mentions_statement','mentions_invoice',
#                    'mentions_fraud','bank_detail_loc','has_signature',
#                     'subject','email_text_body', # text based fields
#                     'true_positives']] # outcome

# email_text_body = ['Good morning, please find our updated bank details attached. Best, Karen',
#                   'Hello, we have updated our bank details. Please find the details attached in the letter.',
#                   'Dear customer, we have changed our bank. Please find our new bank details below.',
#                    'Good Morning Please can you confirm Settlement date of our 2 invoices That you have updated our bank details to the new ones supplied (as attached)',
#                   'Hi Sarah, we have changed bank since last month. Could you please update our information according to the attached letter? Thanks.',
#                   'Hi Moses, we have switched banks and we would kindly ask for our details to be updated. Please find the letter attached here. Best wishes,Chloe',
#                   'Attention: We have updated our bank details. Please find the updated information attached to this email.',
#                    'Good afternoon, we wanted to let you know that we have changed our bank. Please see the new information below:',
#                    'Dear valued customer, please note that our bank information has changed. Attached is a form to update your payment information.',
#                    'Hi Angela, we have updated our bank information. Attached is a document with the new details.',
#                    'This email is to notify you that we have changed our banking information. Please find the updated information attached.',
#                    'Please find your invoice attached. We will never ask you to change your bank details. Please contact us if you receive an email notifying you of any change.',
#                   'Supplied update. Fraud/cybercrime warning: we will never notify you of bank detail changes via email. Please let us know if you receive any email talking about this.',
#                   'Your invoice XY1233 is attached. Please be aware of fraud and vigilant. We will never ask you to change our bank details.',
#                   'Dear customer, we would like to hereby send you your statement. Please respond to the statement as fast as possible and pay any outstanding invoices.',
#                   'You are being contacted as part of our customer base. We want to make you aware of a recent fraud attempt regarding bank details. Please let us know if you have received a potentially fraudulent email.',
#                    'Attention: We have received reports of fraudulent emails regarding bank detail changes. Please disregard any such emails and contact us immediately if you receive one.',
#                    'Please find your updated invoice attached. Please note that we will never ask you to provide your bank details via email.',
#                    'Your statement is attached. Please review it and let us know if you have any questions or concerns. Please also note that we will never ask you to provide sensitive information via email.',
#                    'Dear valued customer, we wanted to make you aware of recent fraud attempts regarding bank details. Please be vigilant and contact us immediately if you receive any suspicious emails.',
#                    'Good morning, please find your order confirmation attached. Please note that we will never ask you to provide your bank details or any other sensitive information via email.',
#                    'Hello Francisco.  If you are not the correct person to provide this information, please forward this request to the appropriate individual.  I am in receipt of the invoices for the vehicles that our station KGTV, ABC 10 News, are purchasing form your dealership.  During the pandemic, AP is working remotely and is not printing manual checks for overnighting.  All of our checks are printed by a 3 rd party and mailed by them.  In order to expedite payment for the vehicles, we are able to pay by ACH, an electronic funds transfer.  Please have your finance department complete the enclosed EFT form and return it to us today, or have them provide one that they already have prepared today, and the payment can be included in tomorrow''s payment run and you should have the funds in your bank account on Friday. Please let me know if you need anything additional from myself or my team here at the corporate AP office Thanks.'
#                   ]

# true_positives = [1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0]

# my_dict = {'email_text_body':email_text_body,'true_positives':true_positives}
# df = pd.DataFrame(my_dict)

## Dummy data to train model on

In [13]:
# target_email_examples = [
    
#     ### True positive examples
#     # short human-like emails
#     'Good morning, please find our updated bank details attached.',
#     'Hello, we have updated our bank details.',
#     'Hi Sarah, we have changed bank since last month.',
#     'Hi Moses, we have switched banks and we would kindly ask for our details to be updated.',
#     'Hi Angela, we have updated our bank information.',
#     'Hello Kareem, we have switched banks and now have new bank information.',
#     'Hello Stephanie,  Please find attached our bank letter with our bank details, please let us know if you need anything else.',
#     'Hi Ryan, Our bank details changed in May 2021!',
#     'Please find attached a letter containing new contact details as there has been an Aquaid branch, and your account has been transferred to Aquaid (South Coast).',
#     'As part of this change I would  be very grateful if you could please also change the bank account details you use to pay all Aquaid invoices to the new bank account given in the letter.',
#     'Good morning, please find attached our updated bank details.',
#     'Hello, we have switched to a new bank.',
#     'Please find attached our bank letter with updated information.',
#     'Good morning, please find attached our updated bank details.',
#     'Hi Joseph, we have updated our bank information.',
#     'Hi Renee, please see the attached document for our new bank information.',
#     'Hi Zahab, attached is a document with our new bank details.',
#     'Hello India, we have switched banks.',
#     'Hello Francis, please find attached our bank letter with updated bank details.',
#     'Hi Aaron, our bank details have changed.',
#     'Please find attached the letter with new bank details as there has been an Aquaid branch transfer.',
#     'Please update our vendor profile with our new bank information.',
#     'We have changed banks and would like you to pay invoices into the new account as the old one is closing soon.',
    
#     # updating more generic
#     'Could you please update our bank information according to the attached letter?',
    
#     # more generic emails
#     'Dear customer, we have changed our bank.',
#     'Please find our new bank details below.',
#     'This email is to notify you that we have changed our banking information.',
#     'Attention: We have updated our bank details.',
#     'Good afternoon, we wanted to let you know that we have changed our bank.',
#     'Dear valued customer, please note that our bank information has changed.',
#     'Please note we have new bank details so please update your systems accordingly',
#     'Please would you arrange for all future BACS and CHAPS payments to be forwarded to this account',
#     'We have changed banks as of 11/14/22.',
#     'Please find attached our new bank details',
#     'Can you please send me a form so I can give you our new bank account information?',
#     'Good morning, Please find the attached requested signed letter in regards to our recent change of bank details.',
#     'Would you be able to help me update our vendor profile along with our bank information?',
#     'We would ask that you now pay invoices into the Action24 bank account as the Wilson one is due to close shortly.',
#     'Dear Sir/Madam  Please find attached Active Security Group Ltd, Re Branding Letter, which also has our New Bank Details.',
#     'However, we are requesting you update our banking details in your system to that of our Ahead, Inc. bank account with Regions Bank.',
#     'The Ahead, Inc. Regions Bank account information is on page 2 of the attachment within a letter from Regions Bank certifying the account.',
#     'OUR BANK DETAILS HAVE RECENTLY CHANGED.',
#     'OUR BANK DETAILS HAVE CHANGED RECENTLY.',
#     'Please see the attached statement for new bank information.',
#     'PLEASE SEE RECENT STATEMENT / INVOICES SHOWING OUR NEW BANK DETAILS.',
#     'NEW BANK DETAILS:',
#     '*** PLEASE NOTE OUR NEW BANK DETAILS WITH IMMEDIATE AFFECT ***',
#     'Please find below a formal letter stating our new company details and our NEW bank account.',
#     'Please ensure all future payments are made to our new Starling Account',
#     'We recently mailed out a notification regarding a change of banking for your lease(s) with InSite, which were acquired by American Tower in March 2021.',
#     'Good Afternoon, We sent out a letter in October regarding a banking change for leases that American Tower acquired from IWG/InSite last year.',
#     'Our records show that E. W. Scripps is still sending payments to our old bank account',
#     'Please note our bank details have recently changed, these can be found at the bottom of the attachment.',
#     'Going forward can you please make all payments to our new AIB bank account, details of our AIB bank account are attached.',
#     'Dear Sir/Madam, please find attached our Rebranding Letter with new bank details for Active Security Group Ltd.',
#     'We have changed our bank account to the Ahead, Inc. account with Regions Bank.',
#     'The bank information is in the attached letter.',
#     'New Account Number: 31354760 New Sort Code: 40-11-60',
    
#     #complicated
#     'As you will be aware, we have merged with Fournet, and as of the 1st of July, our financial administration will change.',
    
#     # bank-change combo with another topic
#     'Good Morning Please can you confirm Settlement date of our 2 invoices that you have updated our bank details to the new ones supplied (as attached)',
#     'Please note: Due to our current banking provider (Ulster Bank) leaving the Irish Market our bank details have changed.',
#     'Our New Bank Details (AIB) are noted at the top of the invoice.',
#     'Please note our new bank details on the invoice.',

# ]
# print(len(target_email_examples))

In [14]:
# neg_email_examples = [
#     ### False negative examples
#     # short human-like emails
#     'Please find attached an updated statement for the Coworth Park account',
#     'I believe there were some queries on deliveries previously but I can''t seem to find too much throughout my emails so I''ve no idea what has happened.',
    
#     # fraud/cybercrime warnings
#     'Please contact us if you receive an email notifying you of any change.',
#     'Fraud/cybercrime warning: we will never notify you of bank detail changes via email.',
#     'We want to make you aware of a recent fraud attempt regarding bank details.',
#     'Please let us know if you have received a potentially fraudulent email.',
#     'Attention: We have received reports of fraudulent emails regarding bank detail changes.',
#     'Please disregard any such emails and contact us immediately if you receive one.',
#     'Dear valued customer, we wanted to make you aware of recent fraud attempts regarding bank details.',
#     'Please be vigilant and contact us immediately if you receive any suspicious emails.',
#     'Please be aware of potential phishing attempts and do not click on suspicious links.',
#     'We would like to emphasize that we will never ask you to disclose your personal information via email.',
#     'Fraud alert: we have received reports of fraudulent emails posing as our company.',
#     'Please be wary of emails asking for gift card payments or wire transfers.',
#     'Please be cautious of emails that request urgent action or threaten consequences.',
#     'Attention: we have noticed an increase in fraudulent emails posing as our company.',
#     'Please be vigilant.',
    
    
#     # Call to action different topics
#     'Please respond to the statement as fast as possible and pay any outstanding invoices.',
#     'Please can you pay your outstanding invoices as soon as possible.',
#     'Please be extra vigilant when opening attachments or clicking links.',
#     'Please consider the environment before printing this email message.',
#     'We kindly request if you could arrange payment as soon as possible?',
#     'We would ask you to focus, in any fresh text that you may care to send us, on your private dining room and its capacity/ambience as this is the information that visitors to are specifically looking for.',
#     'Please additionally advise us of your opening hours every day for lunch and dinner as well as advising us of the number of seated dining guests and standing event guests that your private dining room(s) can  accommodate to enable us to add this information to your listing.',
#     'We would ask that any new content you send us is original, not to be found elsewhere on the internet, as Google favours original content.',
#     'We are waiting to send your first purchase order from our Scripps location KOAA, but first we need some information from you.',
#     'If you are interested in this payment method, please contact US Bank (our bank provider) at 1-855-268-5386, or by email to paymentplus@access-online.com ',
#     'Good Afternoon,  I am following up on our invoice 409292 for PS314.88 that is now overdue for payment',
#     'Please could you confirm if this is now approved and when we can expect to receive payment?',
#     'Please confirm your attendance for the upcoming conference by the end of the week.',
#     'If you have not already done so, please submit your application by the deadline.',
#     'We would appreciate your feedback on our recent product launch. Please take a moment to complete our survey.',
#     'Please visit our website for more information on our services and pricing.',
#     'We encourage you to share your experience with us on social media.',
#     'Please consider making a donation to support our charitable cause.',
#     'Please RSVP for the company holiday party by December 1st.',
#     'If you have any questions or concerns, please do not hesitate to contact us.',
#     'Please save the date for our upcoming charity fundraiser on March 31st.',
#     'PLEASE NOTE OUR BANK DETAILS FOR OUR STERLING ACCOUNT ARE AS BELOW: ROYAL BANK OF SCOTLAND SORT CODE 16-26-32 ACCOUNT NUMBER 11526700 If you have any questions regarding your statement please contact us using the details below',

#     # other types of updating
#     'Please update your contact information in our database to ensure you receive our latest updates.',
#     'Please find your updated invoice attached.',
    
#     # negative change
#     'The Company have not changed their bank details',
#     'ArtSystems have not changed their bank details',
#     'Please note that we will never ask you to provide your bank details via email.',
#     'We will never ask you to change our bank details.',
#     'Please note that we will never ask you to provide your bank details or any other sensitive information via email.',
#     'Should you receive any emails from me regarding a change in bank details, please call the office to check the validity of the request prior to making any change to your systems',
#     'We want to remind you that we will never ask for your password via email.',
#     'Please note that we will never ask for your credit card details over email.',
#     'We want to stress that we will never request sensitive information via email.',
#     'Please do not change the bank details and continue to use the account below for all payments to us.',
#     'Please also note that we will never ask you to provide sensitive information via email.',
    
#     # other topics, e.g. invoices, statements
#     'Dear customer, we would like to hereby send you your statement.',
#     'Your statement is attached. Please review it and let us know if you have any questions or concerns.',
#     'Good morning, please find your order confirmation attached.',
#     'Dear ACCOUNTS PAYABLE, Please find attached a statement of your account',
#     'Hello, Invoice 21110019589 paid 9/23 via EFT/ACH',
#     'This statement is correct as at 01/06/2022',
#     'The attached statement is accurate as of 29/12/2023',
#     'Dear Accounts, On checking your account, we see that you have PS238.17 unpaid from April and prior, details attached.',
    
#     # completely different emails 
#     'I currently don''t see a payment for $1100.00 for the month of March',
#     'Remittance below.',
#     'If you are not the intended recipient, please delete this e-mail immediately',
#     'In order for you or your company to be added to our Procure-to-Pay system, we will need you to complete our Supplier Self-Registration',
#     'Dear customer, Per the attached final audit we all agreed that the credits listed below were correct since you had not paid the invoices',
#     'Today I downloaded the details for the payment processed on 27th Transaction ref.# 287447',
#     'Now, you need to pay back the amount of $21,287.57',
#     'Activity and use of our systems is monitored to secure its effective use and operation and for other lawful business purposes',
#     'Communications using these systems will also be monitored and may be recorded to secure effective use and operation and for other lawful business purposes',
#     'If you are not the addressee, please notify the sender immediately by return e-mail and delete this message',
#     'Company and its subsidiaries',
#     'All product and company names referenced herein are trademarks of their respective owners',
#     'Limited is registered in England and Wales',
#     'The Company is registered in England and Wales',
#     'This ensures the smooth running of all accounts going into the New Year',
#     'Please find attached the report on our company''s sustainability efforts.',    
#     'Please see attached for our latest white paper on industry trends.',
#     'However, we have two other payment options that will allow you to get your payments',
#     'The two invoices expired still not paid are nr 23 and 38 issued by Company on Feb/march 2022, see them attached Amounts are in EUR and we should receive the bank transfer on our Country bank account',
#     'Good morning Stephanie, I have just checked our bank account again and the EFT has not posted and is not pending in our account(which normally would not take this long if you said it was to pay on 5/13/22)',
#     'The above invoices were paid on the payment date above and will be transferred to your bank account via Bacs / Swift',
#     'This change reflects a positive shift in our capabilities and services, and all current staff remain in place',
#     'We believe the most important details, though, are yours - and to keep the information you share with us as safe as possible, we''ve updated our privacy policy in line with the requirements set out by the General Data Protection Regulation',
#     'Additional information regarding your bill, individual charge service details, and your account history are available on the Billing & Cost Management Page.',
    
#     # uncertain about whether to include in training
#     '**Please note that we now take Credit/Debit Cards for payment and we have a Direct Debit facility that all our Clients can sign up to',
#     'Details are shown in the Subject line',
#     'The AWS Customer Agreement was updated on April 30, 2019',
#     'You can see more information about these changes at /agreement/recent-changes/',
    
#     # new additions
# #     'Bank Details Euro Sterling Name: A M Dunne & Son Ltd',
#     'Click here for our bank details',
# #     'Yes, we would change all payments to EFT',
#     'It is against Chess Valley as the supplier but they have new bank details as they have been bought out',
    
# ]

# print(len(neg_email_examples))


# Examples V2

Updates made to the training set:

- took out all greetings (e.g. "Good morning", "Hello Jane", "Hi Alex")
- Made training set more distinct (sentences too similar was harming model performance) e.g. target sentence examples emphasize on "new bank details", e.g. "see our new bank details on the attached invoice", negative examples instead also include "click here for our bank details", "the bank details are on the invoice"

Outstanding question --> We could make it even more sensitive?

In [13]:
target_email_examples = [
    
    ### True positive examples
    # short human-like emails
    'please find our updated bank details attached.',
    'we have updated our bank details.',
    'we have changed bank since last month.',
    'we have switched banks and we would kindly ask for our details to be updated.',
    'we have updated our bank information.',
    'we have switched banks and now have new bank information.',
    'Please find attached our bank letter with our bank details, please let us know if you need anything else.',
    'Our bank details changed in May 2021!',
    'Please find attached a letter containing new contact details as there has been an Aquaid branch, and your account has been transferred to Aquaid (South Coast).',
    'As part of this change I would be very grateful if you could please also change the bank account details you use to pay all Aquaid invoices to the new bank account given in the letter.',
    'please find attached our updated bank details.',
    'we have switched to a new bank.',
    'Please find attached our bank letter with updated information.',
    'please find attached our updated bank details.',
    'we have updated our bank information.',
    'please see the attached document for our new bank information.',
    'attached is a document with our new bank details.',
    'we have switched banks.',
    'please find attached our bank letter with updated bank details.',
    'our bank details have changed.',
    'Please find attached the letter with new bank details as there has been an Aquaid branch transfer.',
    'Please update our vendor profile with our new bank information.',
    'We have changed banks and would like you to pay invoices into the new account as the old one is closing soon.',
    
    # updating more generic
    'Could you please update our bank information according to the attached letter?',
    
    # more generic emails
    'we have changed our bank.',
    'Please find our new bank details below.',
    'This email is to notify you that we have changed our banking information.',
    'Attention: We have updated our bank details.',
    'we wanted to let you know that we have changed our bank.',
    'please note that our bank information has changed.',
    'Please note we have new bank details so please update your systems accordingly',
    'Please would you arrange for all future BACS and CHAPS payments to be forwarded to this account',
    'We have changed banks as of 11/14/22.',
    'Please find attached our new bank details',
    'Can you please send me a form so I can give you our new bank account information?',
    'Please find the attached requested signed letter in regards to our recent change of bank details.',
    'Would you be able to help me update our vendor profile along with our bank information?',
    'We would ask that you now pay invoices into the Action24 bank account as the Wilson one is due to close shortly.',
    'Please find attached Active Security Group Ltd, Re Branding Letter, which also has our New Bank Details.',
    'However, we are requesting you update our banking details in your system to that of our Ahead, Inc. bank account with Regions Bank.',
    'The Ahead, Inc. Regions Bank account information is on page 2 of the attachment within a letter from Regions Bank certifying the account.',
    'OUR BANK DETAILS HAVE RECENTLY CHANGED.',
    'OUR BANK DETAILS HAVE CHANGED RECENTLY.',
    'Please see the attached statement for new bank information.',
    'PLEASE SEE RECENT STATEMENT / INVOICES SHOWING OUR NEW BANK DETAILS.',
    'NEW BANK DETAILS:',
    '*** PLEASE NOTE OUR NEW BANK DETAILS WITH IMMEDIATE AFFECT ***',
    'Please find below a formal letter stating our new company details and our NEW bank account.',
    'Please ensure all future payments are made to our new Starling Account',
    'We recently mailed out a notification regarding a change of banking for your lease(s) with InSite, which were acquired by American Tower in March 2021.',
    'We sent out a letter in October regarding a banking change for leases that American Tower acquired from IWG/InSite last year.',
    'Our records show that E. W. Scripps is still sending payments to our old bank account',
    'Please note our bank details have recently changed, these can be found at the bottom of the attachment.',
    'Going forward can you please make all payments to our new AIB bank account, details of our AIB bank account are attached.',
    'please find attached our Rebranding Letter with new bank details for Active Security Group Ltd.',
    'We have changed our bank account to the Ahead, Inc. account with Regions Bank.',
    'The bank information is in the attached letter.',
    'New Account Number: 31354760 New Sort Code: 40-11-60',
    
    #complicated
    'As you will be aware, we have merged with Fournet, and as of the 1st of July, our financial administration will change.',
    
    # bank-change combo with another topic
    'Please can you confirm Settlement date of our 2 invoices that you have updated our bank details to the new ones supplied (as attached)',
    'Please note: Due to our current banking provider (Ulster Bank) leaving the Irish Market our bank details have changed.',
    'Our New Bank Details (AIB) are noted at the top of the invoice.',
    'Please note our new bank details on the invoice.',
    'Please note that our bank details have changed, see attached',
    'Can you amend your records for all future payments',
    'Can you please make this change to our account on your systems with immediate effect for all future payments',
    'As part of this change I would be very grateful if you could please also change the bank account details you use to pay all Aquaid invoices to the new bank account given in the letter',

]
print(len(target_email_examples))

67


In [14]:
neg_email_examples = [
    ### False negative examples
    # short human-like emails
    'Please find attached an updated statement for the Coworth Park account',
    
    # fraud/cybercrime warnings
    'Please contact us if you receive an email notifying you of any change.',
    'Fraud/cybercrime warning: we will never notify you of bank detail changes via email.',
    'We want to make you aware of a recent fraud attempt regarding bank details.',
    'Please let us know if you have received a potentially fraudulent email.',
    'Attention: We have received reports of fraudulent emails regarding bank detail changes.',
    'Please disregard any such emails and contact us immediately if you receive one.',
    'we wanted to make you aware of recent fraud attempts regarding bank details.',
    'Please be vigilant and contact us immediately if you receive any suspicious emails.',
    'Please be aware of potential phishing attempts and do not click on suspicious links.',
    'We would like to emphasize that we will never ask you to disclose your personal information via email.',
    'Fraud alert: we have received reports of fraudulent emails posing as our company.',
    'Please be wary of emails asking for gift card payments or wire transfers.',
    'Please be cautious of emails that request urgent action or threaten consequences.',
    'Attention: we have noticed an increase in fraudulent emails posing as our company.',
    'Please be vigilant.',
    
    # Call to action different topics
    'Please respond to the statement as fast as possible and pay any outstanding invoices.',
    'Please can you pay your outstanding invoices as soon as possible.',
    'Please be extra vigilant when opening attachments or clicking links.',
    'Please consider the environment before printing this email message.',
    'We kindly request if you could arrange payment as soon as possible?',
    'We would ask you to focus, in any fresh text that you may care to send us, on your private dining room and its capacity/ambience as this is the information that visitors to are specifically looking for.',
    'Please additionally advise us of your opening hours every day for lunch and dinner as well as advising us of the number of seated dining guests and standing event guests that your private dining room(s) can  accommodate to enable us to add this information to your listing.',
    'We would ask that any new content you send us is original, not to be found elsewhere on the internet, as Google favours original content.',
    'We are waiting to send your first purchase order from our Scripps location KOAA, but first we need some information from you.',
    'If you are interested in this payment method, please contact US Bank (our bank provider) at 1-855-268-5386, or by email to paymentplus@access-online.com ',
    'I am following up on our invoice 409292 for PS314.88 that is now overdue for payment',
    'Please could you confirm if this is now approved and when we can expect to receive payment?',
    'Please confirm your attendance for the upcoming conference by the end of the week.',
    'If you have not already done so, please submit your application by the deadline.',
    'We would appreciate your feedback on our recent product launch. Please take a moment to complete our survey.',
    'Please visit our website for more information on our services and pricing.',
    'We encourage you to share your experience with us on social media.',
    'Please consider making a donation to support our charitable cause.',
    'Please RSVP for the company holiday party by December 1st.',
    'If you have any questions or concerns, please do not hesitate to contact us.',
    'Please save the date for our upcoming charity fundraiser on March 31st.',
    'PLEASE NOTE OUR BANK DETAILS FOR OUR STERLING ACCOUNT ARE AS BELOW: ROYAL BANK OF SCOTLAND SORT CODE 16-26-32 ACCOUNT NUMBER 11526700 If you have any questions regarding your statement please contact us using the details below',

    # other types of updating
    'Please update your contact information in our database to ensure you receive our latest updates.',
    'Please find your updated invoice attached.',
    
    # negative change
    'The Company have not changed their bank details',
    'ArtSystems have not changed their bank details',
    'Please note that we will never ask you to provide your bank details via email.',
    'We will never ask you to change our bank details.',
    'Please note that we will never ask you to provide your bank details or any other sensitive information via email.',
    'Should you receive any emails from me regarding a change in bank details, please call the office to check the validity of the request prior to making any change to your systems',
    'We want to remind you that we will never ask for your password via email.',
    'Please note that we will never ask for your credit card details over email.',
    'We want to stress that we will never request sensitive information via email.',
    'Please do not change the bank details and continue to use the account below for all payments to us.',
    'Please also note that we will never ask you to provide sensitive information via email.',
    
    # other topics, e.g. invoices, statements
    'we would like to hereby send you your statement.',
    'Your statement is attached. Please review it and let us know if you have any questions or concerns.',
    'please find your order confirmation attached.',
    'Dear ACCOUNTS PAYABLE, Please find attached a statement of your account',
    'Invoice 21110019589 paid 9/23 via EFT/ACH',
    'This statement is correct as at 01/06/2022',
    'The attached statement is accurate as of 29/12/2023',
    'On checking your account, we see that you have PS238.17 unpaid from April and prior, details attached.',
    
    # completely different emails 
    'I currently don''t see a payment for $1100.00 for the month of March',
    'Remittance below.',
    'If you are not the intended recipient, please delete this e-mail immediately',
    'In order for you or your company to be added to our Procure-to-Pay system, we will need you to complete our Supplier Self-Registration',
    'Per the attached final audit we all agreed that the credits listed below were correct since you had not paid the invoices',
    'Today I downloaded the details for the payment processed on 27th Transaction ref.# 287447',
    'Now, you need to pay back the amount of $21,287.57',
    'Activity and use of our systems is monitored to secure its effective use and operation and for other lawful business purposes',
    'Communications using these systems will also be monitored and may be recorded to secure effective use and operation and for other lawful business purposes',
    'If you are not the addressee, please notify the sender immediately by return e-mail and delete this message',
    'Company and its subsidiaries',
    'All product and company names referenced herein are trademarks of their respective owners',
    'Limited is registered in England and Wales',
    'The Company is registered in England and Wales',
    'This ensures the smooth running of all accounts going into the New Year',
    'Please find attached the report on our company''s sustainability efforts.',    
    'Please see attached for our latest white paper on industry trends.',
    'However, we have two other payment options that will allow you to get your payments',
    'The two invoices expired still not paid are nr 23 and 38 issued by Company on Feb/march 2022, see them attached Amounts are in EUR and we should receive the bank transfer on our Country bank account',
    'I have just checked our bank account again and the EFT has not posted and is not pending in our account(which normally would not take this long if you said it was to pay on 5/13/22)',
    'The above invoices were paid on the payment date above and will be transferred to your bank account via Bacs / Swift',
    'This change reflects a positive shift in our capabilities and services, and all current staff remain in place',
    'We believe the most important details, though, are yours - and to keep the information you share with us as safe as possible, we''ve updated our privacy policy in line with the requirements set out by the General Data Protection Regulation',
    'Additional information regarding your bill, individual charge service details, and your account history are available on the Billing & Cost Management Page.',
    
    # uncertain about whether to include in training
    '**Please note that we now take Credit/Debit Cards for payment and we have a Direct Debit facility that all our Clients can sign up to',
    'Details are shown in the Subject line',
    'The AWS Customer Agreement was updated on April 30, 2019',
    'You can see more information about these changes at /agreement/recent-changes/',
    
    # new additions
#     'Bank Details Euro Sterling Name: A M Dunne & Son Ltd',
    'Click here for our bank details',
#     'Yes, we would change all payments to EFT',
#     'It is against Chess Valley as the supplier but they have new bank details as they have been bought out',
    
    'Kindly update your systems/records with our new email address details: ',
    'The bank details are the same, however, invoices have different numbers',
    'will be updating some of our email addresses and as such the email address from which you receive your invoices and statements from us will change',
    'The bank details are still the same',
    'The bank details have not changed',
    'There are no changes to the bank details',
    'Further to our email communication sent last month, we can now confirm the legal name change of Dorma UK Ltd to dormakaba UK Ltd',
    'What is the new companies house number',
    'In continuing our continuous improvement plan, we are upgrading our core computer systems to SAP',
    'We''ve rebranded',
    'The new and updated Pierre Frey website now provides you with more information on dimensions, FR rating, composition and more',
    'The assets of Map Marketing Ltd have been purchased by a new company All Jigsaw Puzzles Ltd and the factory in the UK with its existing staff will continue to manufacture the same products as before, without any interruption to production',
    'WE HAVE MOVED 1 st floor, 1 Europa Drive, Sheffield, S9 1XT',
    'In case new linked accounts are added, click ''Enable Tax Settings Inheritance'' so the tax registration information will automatically be inherited from your Payer account to your Linked account',
    'If you can please reply and confirm as per the attached statement that these invoices will be paid as per the agreed terms we can get the new orders dispatched',
    'We constantly look at ways to improve the service we offer, and as part of this we have moved to utilise a new system called Sidetrade which will deliver improvements in the way we communicate with you',
    'I understand that you require our full data to set us up on your system as a supplier, so could you please send me your ''New Suppliers Account'' form if applicable',
    'As of August, Bottomline Technologies will be updating some of our email addresses and as such the email address from which you receive your invoices and statements from us will change',
    'We are writing to you as an existing customer of Wilson Security to let you know that as of 22nd December 2022 Action Alarms Limited t/a Action24 acquired Wilson Security Limited',
    'The Dorma name change does not yet mark the full corporate merge of the two businesses',
    'The assets of Map Marketing Ltd have now been purchased by a new company ''All Jigsaw Puzzles Ltd'' that has taken over the trade accounts (including yours) and the factory in Hatherleigh with its existing staff and will continue to manufacture the same map based goods, laminated planners and',
    'Change of Name/ Brand/ Trading Style As you are aware, Irongate Group was acquired by Complete Business Solutions Group Ltd on the 1st November 2018 and Irongate Group became a trading name of Complete Business Solutions Group Ltd',
    'The template for our Statements and Dunning correspondence will change',
    'Direct Debit payment collections dates will now be in line with the due date listed on our Invoices',
    'The name change is a consequence of our rebrand to position us better in the market in line with the products and service we now offer',
    'Please accept this communication by way of informing you that we have recently changed our registered company name from ACS Business Supplies Ltd to ACS Technology Group Ltd',
    'As part of our ongoing commitment to invest in innovation and continuous improvement, towards the end of last year, we announced a business restructure, with Swift360 joining the SMI Group family',
    'From 15th August 2022, all employees will be physically located in the same office, with Swift360 team members joining the SMI team in our new head office in Petersfield, Hampshire',
    'We converted to a new business model as at the close of play yesterday (we''re now a "John Lewis" style of ownership) and I''m afraid our accounts department is completely overrun with the switch over',
    
    
    
]

print(len(neg_email_examples))


117


In [15]:
email_bodies = target_email_examples + neg_email_examples

pos = [1]*len(target_email_examples)
negs = [0]*len(neg_email_examples)

true_positives = pos + negs

my_dict = {'email_text_body':email_bodies,'true_positives':true_positives}
df = pd.DataFrame(my_dict)
print(len(df))

184


In [16]:
df

,email_text_body,true_positives
0,please find our updated bank details attached.,1
1,we have updated our bank details.,1
2,we have changed bank since last month.,1
3,we have switched banks and we would kindly ask...,1
4,we have updated our bank information.,1
5,we have switched banks and now have new bank i...,1
6,Please find attached our bank letter with our ...,1
7,Our bank details changed in May 2021!,1
8,Please find attached a letter containing new c...,1
9,As part of this change I would be very gratefu...,1


In [19]:
# select 
subset = df.loc[:, ['email_text_body','true_positives']] # outcome

subset = subset.rename(columns={"email_text_body":"text","true_positives":"label"})
subset["label_text"] = subset.apply(lambda row: "positive" if row["label"] else "negative", axis = 1)

# randomly sample x from dataset
# neg_ex = subset.query("label == 0").sample(n=10,random_state=42).reset_index(drop=True)
# pos_ex = subset.query("label == 1").sample(n=10,random_state=42).reset_index(drop=True)

neg_ex = subset.query("label == 0").reset_index(drop=True)
pos_ex = subset.query("label == 1").reset_index(drop=True)

# combine datasets and shuffle them
this_df = (pd.concat([neg_ex, pos_ex])).sample(frac=1)

print(len(this_df))
# print(this_df.head(10))
# del subset

184


# Train the SetFit model

In [23]:
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitModel, SetFitTrainer
from sentence_transformers import SentenceTransformer
import huggingface
from sklearn.model_selection import train_test_split
import datasets
import torch

In [21]:
# split into train and test set
train_data, test_data = train_test_split(this_df, test_size=0.25, random_state=42)

train_ds = datasets.Dataset.from_dict(train_data.to_dict(orient="list"))
test_ds = datasets.Dataset.from_dict(test_data.to_dict(orient="list"))

In [22]:
print(len(train_ds))
print(len(test_ds))

138
46


In [15]:
# Load a setfit model from the hub
model_name = "sentence-transformers/paraphrase-mpnet-base-v2"

In [16]:
model = SetFitModel.from_pretrained(model_name)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [42]:
dir(model)

['__call__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_freeze_or_not',
 '_from_pretrained',
 '_prepare_dataloader',
 '_prepare_optimizer',
 '_save_pretrained',
 'create_model_card',
 'fit',
 'freeze',
 'from_pretrained',
 'has_differentiable_head',
 'l2_weight',
 'model_body',
 'model_head',
 'multi_target_strategy',
 'normalize_embeddings',
 'predict',
 'predict_proba',
 'push_to_hub',
 'save_pretrained',
 'to',
 'unfreeze']

In [45]:
print(torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))

cuda


In [47]:
model.to("cuda")

SetFitModel()

In [48]:
# Create trainer - add early stopping
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    loss_class=CosineSimilarityLoss,
    batch_size=16,
    num_iterations=20, # Number of text pairs to generate for contrastive learning
    num_epochs=5 #1 #5, # Number of epochs to use for contrastive learning
)

In [49]:
%%time
# after instantiating the trainer, train it
trainer.train()
metrics = trainer.evaluate()
metrics

***** Running training *****
  Num examples = 5520
  Num epochs = 5
  Total optimization steps = 1725
  Total train batch size = 16


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/345 [00:00<?, ?it/s]

Iteration:   0%|          | 0/345 [00:00<?, ?it/s]

Iteration:   0%|          | 0/345 [00:00<?, ?it/s]

Iteration:   0%|          | 0/345 [00:00<?, ?it/s]

Iteration:   0%|          | 0/345 [00:00<?, ?it/s]

***** Running evaluation *****


CPU times: user 10min 55s, sys: 4min, total: 14min 55s
Wall time: 14min 47s


{'accuracy': 1.0}

### Quick summary

Model reaches 0.974% accuracy on training data (highest I could get it)

In [ ]:
# metrics._logits

In [ ]:
# logits = torch.from_numpy(metrics["eval_logits"])

setfit has this class method

model._save_pretrained(save_directory)

and to load it

saved_model = SetFitModel._from_pretrained(save_directory)


In [50]:
# save model
model._save_pretrained('trained_models/SetFit_Trained4')

In [24]:
# load model
model = SetFitModel._from_pretrained('trained_models/SetFit_Trained4')

Save and load models: https://stackoverflow.com/questions/74046426/how-to-save-a-setfit-trainer-locally-after-training

save with:

import pickle
import joblib

- pickle.dump(trainer, open('model.pkl', 'wb'))
- joblib.dump(trainer, 'model.joblib')

load with:

- job_model = joblib.load('model.joblib')
- pkl_model = pickle.load(open('model.pkl', 'rb'))

In [50]:
# import pickle
# import joblib

# pickle.dump(trainer, open('trained_models/setfit_80dp_1303_V1.pkl', 'wb'))
# joblib.dump(trainer, 'trained_models/setfit_80dp_1303_V1.joblib')

In [51]:
# # Calculate the recall for each class
# for i, label in enumerate(test_ds.unique('label')):
#     tp = eval_results.confusion_matrix[i, i]
#     fn = eval_results.confusion_matrix[i, :].sum() - tp
#     recall = tp / (tp + fn)
#     print(f"Recall for class {label}: {recall:.3f}")

# Select real data - 650 rows of manually labelled data by Claire

In [12]:
# # # import cleaned data
# df = pd.read_csv("data/checked_emails.csv")
# df = df.rename(columns={'bank-detail-location': 'bank_detail_loc'})
# df.head()

# # filter on the ones that are clear cases
# df_clear = df.query("uncertain == 0").reset_index(drop=True)

# # # select 
# # subset = df.loc[:, ['has_sort_code', 'mentions_statement','mentions_invoice',
# #                    'mentions_fraud','bank_detail_loc','has_signature',
# #                     'subject','email_text_body', # text based fields
# #                     'true_positives']] # outcome


# # select 
# subset = df_clear.loc[:, ['email_text_body', # text based fields
#                     'true_positives']] # outcome

# subset = subset.rename(columns={"email_text_body":"text","true_positives":"label"})
# subset["label_text"] = subset.apply(lambda row: "positive" if row["label"] else "negative", axis = 1)

# # randomly sample x from dataset
# neg_ex = subset.query("label == 0").sample(n=100,random_state=42).reset_index(drop=True)
# pos_ex = subset.query("label == 1").sample(n=100,random_state=42).reset_index(drop=True)

# # combine datasets and shuffle them
# this_df = (pd.concat([neg_ex, pos_ex])).sample(frac=1)

# print(len(this_df))
# this_df.head(10)

In [13]:
# # # import cleaned data
# # df = pd.read_csv("data/checked_emails.csv")
# df = pd.read_csv("data/Target_emails_V2.csv")
# df = df.rename(columns={'bank-detail-location': 'bank_detail_loc'})
# df.head()

# # filter on the ones that are clear cases
# df_clear = df.query("final_label != -1").reset_index(drop=True)

# # select 
# subset = df_clear.loc[:, ['email_text_body', # text based fields
#                     'final_label']] # outcome

# subset = subset.rename(columns={"email_text_body":"text","final_label":"label"})
# subset["label_text"] = subset.apply(lambda row: "positive" if row["label"] else "negative", axis = 1)

# # randomly sample x from dataset
# neg_ex = subset.query("label == 0").sample(n=100,random_state=42).reset_index(drop=True)
# pos_ex = subset.query("label == 1").sample(n=100,random_state=42).reset_index(drop=True)

# # combine datasets and shuffle them
# this_df = (pd.concat([neg_ex, pos_ex])).sample(frac=1)

# print(len(this_df))
# print(this_df.head(10))
# del subset

# New method of sentence-by-sentence

## Select real data - 1001 rows of manually labelled data by James & Claire

Here we

- keep all features from the rest of the data (e.g. the email ID so we can find back emails on that later)
- read in all the emails:
    - first from the rule-based filtered out emails (˜3k):
        - all the manually labelled emails (by me and James) (˜1000)
        - the other emails (˜2000)
    - next the non-filtered out emails (˜400k):
        - batches of 10k?


In [51]:
# # import cleaned data
# df = pd.read_csv("data/checked_emails.csv")
df = pd.read_csv("data/Target_emails_V2.csv")
df = df.rename(columns={'bank-detail-location': 'bank_detail_loc'})
df.head()

# filter on the ones that are clear cases
this_df = df.query("final_label != -1").reset_index(drop=True)

# # select 
# subset = df_clear.loc[:, ['email_text_body', # text based fields
#                     'final_label']] # outcome

# rename some columns
this_df = this_df.rename(columns={"email_text_body":"text","final_label":"label"})
this_df["label_text"] = this_df.apply(lambda row: "positive" if row["label"] else "negative", axis = 1)

this_df = this_df.sample(frac=0.1) # change frac for different sampling rates

print(len(this_df))
perc = (sum(this_df["label"])*100)/len(this_df)
print(f'true positives: {this_df["label"].sum()}, {"{:.1f}".format(perc)}%')
this_df.head(10)

100
true positives: 44, 44.0%


,Index,iter_count_list,email_id,is_part_of_thread,num_in_thread,thread_id,initial_domain,from_address,from_name,sent_to,sent_to_how_many,bcc,cc,reply_to_address,internal_date,category_id,category_slug,has_sort_code,sort_code,has_attachment,mentions_bank,mentions_change,mentions_account_specifics,mentions_statement,mentions_invoice,mentions_fraud,has_attention_ext_flag,has_alert_attach_flag,subject,attachments,text,label,label_james,true_positives,false_positives,uncertain,email_type,bank_detail_loc,has_signature,label_text
175,26246,16908,458543,True,1,458543_t,dorchestercollection.com,[' GeorginaOToole@matthewclark.co.uk '],Georgina OToole,[' APInvoicequeries.UK@dorchestercollection.co...,1,NaN,NaN,{},09/06/2022 13:25,0,NaN,True,"""20-00-00""",False,True,True,True,False,True,False,False,False,Payment required - Dorchester Hotel,NaN,Customer Number Invoice Date Document Num...,0,0.0,NaN,NaN,0,NaN,NaN,NaN,negative
196,33979,19422,461057,True,1,461057_t,dorchestercollection.com,['samanthamalkin@rhug.co.uk'],Samantha Malkin,['APInvoicequeries.UK@dorchestercollection.com'],1,NaN,NaN,{},01/11/2021 16:02,0,NaN,False,NaN,False,True,True,True,True,False,False,False,False,Rhug Organic - Statement,NaN,Please find attached a current statement of a...,0,0.0,NaN,NaN,0,NaN,NaN,NaN,negative
600,21393,19188,236695,False,0,FALSE,autosmart.uk.net,stephen.willis@autosmart.uk.net,stephen.willis@autosmart.uk.net,{Purchase.Ledger@evo-group.co.uk},1,{},{},{},2022-01-20 12:57:45+00,0,NaN,True,"""40-11-18""",True,True,True,True,False,True,False,True,False,Autosmart CASHSALE 0108524,0108524.pdf,Attention: This email originated outside of ...,1,1.0,1.0,0.0,1,automatic-reply,body,1.0,positive
900,65700,40503,432138,False,0,FALSE,crawfords.co.uk,caroline@crawfords.co.uk,caroline@crawfords.co.uk,{APInvoicequeries.UK@dorchestercollection.com},1,{},{},{},2022-04-26 13:07:47+00,0,NaN,False,NaN,False,True,True,False,False,False,False,False,False,Crawford Cars = Account Number 0333 = Februa...,NaN,Good Afternoon I hope this email finds you all...,1,NaN,1.0,0.0,0,human,attached,1.0,positive
930,75290,45409,437044,False,0,FALSE,decantae.co.uk,admin@decantae.co.uk,admin@decantae.co.uk,"{Invoicepayments.UK@dorchestercollection.com,A...",2,{},{},{},2022-02-09 14:20:09+00,0,NaN,False,NaN,True,True,True,False,False,True,False,False,False,Decantae Mineral Water Ltd Inv No:,Invoice 101609.pdf,Hi The attached credit relates to invoice 9823...,1,NaN,1.0,0.0,0,human,attached,1.0,positive
341,6869,6170,223678,False,0,FALSE,artsystems.co.uk,accounts@artsystems.co.uk,accounts@artsystems.co.uk,{purchase.ledger@evo-group.co.uk},1,{},{},{},2021-12-17 16:45:46+00,0,NaN,False,NaN,True,True,True,False,True,False,False,True,False,ArtSystems Month End Statement,Statement CU-10000037.pdf,Attention: This email originated outside of o...,0,NaN,0.0,1.0,0,automatic-reply,0,1.0,negative
652,22263,13687,455322,True,10,455322_t,dorchestercollection.com,['pwatts@kendraenergy.co.uk'],Paula Watts,"['Jyoti.Hirani@dorchestercollection.com', 'AP....",3,NaN,NaN,{},15/04/2020 08:04,0,NaN,False,NaN,False,True,True,True,True,False,False,False,False,The Dorchester- Outstanding ¬£4104.00,NaN,"Good Morning, I have not received an update...",1,1.0,NaN,NaN,0,NaN,NaN,NaN,positive
685,32678,18996,460631,True,1,460631_t,dorchestercollection.com,['aurelija@balloonista.co.uk'],Aurelija Meakin,['APInvoicequeries.UK@dorchestercollection.com'],1,NaN,NaN,{},29/11/2021 10:18,0,NaN,False,NaN,True,True,True,False,False,False,False,False,False,Balloonista Ltd address and Bank account change,NaN,"Good morning, As discussed earlier, please fi...",1,1.0,NaN,NaN,0,NaN,NaN,NaN,positive
598,10984,9923,227430,False,0,FALSE,aquaid-wilts.co.uk,Alex@aquaid-wilts.co.uk,Alex@aquaid-wilts.co.uk,"{Purchase.Ledger@evo-group.co.uk,rachel.gearon...",2,{},{},{},2022-01-03 14:34:29+00,0,NaN,False,NaN,True,True,True,False,False,False,False,True,False,Ref Swindon Office (BAN002) Aquaid Branch Rest...,Aquaid (Kennet & North Wilts) Branch Me

In [18]:
this_df.columns

Index(['Index', 'iter_count_list', 'email_id', 'is_part_of_thread',
       'num_in_thread', 'thread_id', 'initial_domain', 'from_address',
       'from_name', 'sent_to', 'sent_to_how_many', 'bcc', 'cc',
       'reply_to_address', 'internal_date', 'category_id', 'category_slug',
       'has_sort_code', 'sort_code', 'has_attachment', 'mentions_bank',
       'mentions_change', 'mentions_account_specifics', 'mentions_statement',
       'mentions_invoice', 'mentions_fraud', 'has_attention_ext_flag',
       'has_alert_attach_flag', 'subject', 'attachments', 'text', 'label',
       'label_james', 'true_positives', 'false_positives', 'uncertain',
       'email_type', 'bank_detail_loc', 'has_signature', 'label_text'],
      dtype='object')

### Move this to funtions file

In [25]:
# import re 
# import string

# def clean_websites(text):
#     '''Clean text in following ways:
#     - remove text in square brackets [] and within {}
#     - remove all https links <>
#     - remove line breaks ('\n')
#     - remove "-----Original Message-----"
#     '''
    
#     url_pattern = r'(https?:\/\/\S+?\.(com|co\.uk|org|co|uk))|(www\.\S+?\.(com|co\.uk|org|co|uk))'
#     email_pattern = r'\b\w+@\w+\.\w+\b'
    
#     text = re.sub(url_pattern, '', text) # clean text from within square brackets 
#     text = re.sub(email_pattern, '', text)
    
#     return text

## Loop through the emails sentence-by-sentence

In [26]:
import re 
import string

In [27]:
%%time
email_num = 1

cols = ['this_email','sentence','true_label','true_class','pred_label']
cols_2 = ['this_email','true_label','sum_score','email_length','perc_flagged']


df_list = []
email_list = []

split_sentence_string = "[.?!]"

# change name of df here
for email in this_df.itertuples():


#     sentences = email.text.split('.')
    clean_text = clean_websites(email.text)
    
    sentences = re.split(split_sentence_string,clean_text)
    email_num += 1
    sum_score = 0
    
    len_email = len(sentences)
    
    for s in sentences:
        
        # check if this bit of the list is empty
        if len(s.strip()) == 0:
            continue
        else:

            preds = model([s])
            this_label = preds.item()
            sum_score += this_label

            row_data = [email_num,s,email.label,email.label_text,this_label]
            df_row = pd.DataFrame([row_data],columns = cols)
            df_list.append(df_row)

    # save the email specific info
    perc_flag = "{:.1f}".format((sum_score*100)/len_email)
    
    email_data = [email_num,email.label,sum_score,len_email,perc_flag]
    email_row = pd.DataFrame([email_data], columns = cols_2)
    email_list.append(email_row)
    

# save when this is completed
email_df = pd.concat(df_list, ignore_index=True)
meta_df = pd.concat(email_list, ignore_index=True)

CPU times: user 4min 22s, sys: 5.38 s, total: 4min 27s
Wall time: 1min 7s


In [28]:
meta_df

,this_email,true_label,sum_score,email_length,perc_flagged
0,2,0,0,36,0.0
1,3,1,0,7,0.0
2,4,0,0,32,0.0
3,5,1,1,5,20.0
4,6,1,2,9,22.2
5,7,1,2,12,16.7
6,8,0,0,26,0.0
7,9,1,0,6,0.0
8,10,0,0,8,0.0
9,11,1,2,9,22.2


In [55]:
# function for this loop
def sentence_by_sentence(df, model):
    '''Classify the text of the emails in a sentence-by-sentence manner
    '''
    
    email_iter = 1

    sentence_cols = ['this_email','email_id','is_part_of_thread','num_in_thread','thread_id','initial_domain',
                     'from_address','sentence','true_label','true_class','y_pred']
        
    df_list = []

    # to split the sentences by
    split_sentence_string = "[.?!]"

    # change name of df here
    for email in df.itertuples():
        
        # try/except handling here
        try:

            clean_text = clean_websites(email.text)
            sentences = re.split(split_sentence_string,clean_text)

            # some counters
            email_iter += 1
            sum_score = 0

            len_email = len(sentences)

            for s in sentences:

                # check if this bit of the list is empty
                if len(s.strip()) == 0:
                    continue
                else:

                    preds = model([s])
                    this_label = preds.item()
                    sum_score += this_label

                    # save row data
                    row_data = [email_iter, email.email_id, email.is_part_of_thread, 
                                email.num_in_thread, email.thread_id,
                        email.initial_domain, email.from_address,
                        s,email.label,email.label_text,this_label]

                    df_row = pd.DataFrame([row_data],columns = sentence_cols)
                    df_list.append(df_row)

            ### save email sentence info
            sentence_df = pd.concat(df_list, ignore_index=True)

            ### save the meta email info
            perc_flag = "{:.1f}".format((sum_score*100)/len_email)

            # binary coding 
            y_pred = 1 if sum_score > 0 else 0
            diff = 0 if y_pred == email.label else 1

            # values to append to the big data frame
            df.at[email.Index,'sum_score'] = sum_score
            df.at[email.Index,'len_email'] = len_email
            df.at[email.Index,'perc_flag'] = perc_flag
            df.at[email.Index,'y_pred'] = y_pred
            df.at[email.Index,'diff'] = diff
            
        except:
            print(f'error occurred at email_iter: {email_iter} email_id: {email.email_id} thread_id: {email.thread_id}')

    return sentence_df, df

In [53]:
row_df , meta_df = sentence_by_sentence(this_df, model)

In [54]:
meta_df

,Index,iter_count_list,email_id,is_part_of_thread,num_in_thread,thread_id,initial_domain,from_address,from_name,sent_to,sent_to_how_many,bcc,cc,reply_to_address,internal_date,category_id,category_slug,has_sort_code,sort_code,has_attachment,mentions_bank,mentions_change,mentions_account_specifics,mentions_statement,mentions_invoice,mentions_fraud,has_attention_ext_flag,has_alert_attach_flag,subject,attachments,text,label,label_james,true_positives,false_positives,uncertain,email_type,bank_detail_loc,has_signature,label_text,sum_score,len_email,perc_flag,y_pred,diff
175,26246,16908,458543,True,1,458543_t,dorchestercollection.com,[' GeorginaOToole@matthewclark.co.uk '],Georgina OToole,[' APInvoicequeries.UK@dorchestercollection.co...,1,NaN,NaN,{},09/06/2022 13:25,0,NaN,True,"""20-00-00""",False,True,True,True,False,True,False,False,False,Payment required - Dorchester Hotel,NaN,Customer Number Invoice Date Document Num...,0,0.0,NaN,NaN,0,NaN,NaN,NaN,negative,0.0,47.0,0.0,0.0,0.0
196,33979,19422,461057,True,1,461057_t,dorchestercollection.com,['samanthamalkin@rhug.co.uk'],Samantha Malkin,['APInvoicequeries.UK@dorchestercollection.com'],1,NaN,NaN,{},01/11/2021 16:02,0,NaN,False,NaN,False,True,True,True,True,False,False,False,False,Rhug Organic - Statement,NaN,Please find attached a current statement of a...,0,0.0,NaN,NaN,0,NaN,NaN,NaN,negative,0.0,15.0,0.0,0.0,0.0
600,21393,19188,236695,False,0,FALSE,autosmart.uk.net,stephen.willis@autosmart.uk.net,stephen.willis@autosmart.uk.net,{Purchase.Ledger@evo-group.co.uk},1,{},{},{},2022-01-20 12:57:45+00,0,NaN,True,"""40-11-18""",True,True,True,True,False,True,False,True,False,Autosmart CASHSALE 0108524,0108524.pdf,Attention: This email originated outside of ...,1,1.0,1.0,0.0,1,automatic-reply,body,1.0,positive,1.0,6.0,16.7,1.0,0.0
900,65700,40503,432138,False,0,FALSE,crawfords.co.uk,caroline@crawfords.co.uk,caroline@crawfords.co.uk,{APInvoicequeries.UK@dorchestercollection.com},1,{},{},{},2022-04-26 13:07:47+00,0,NaN,False,NaN,False,True,True,False,False,False,False,False,False,Crawford Cars = Account Number 0333 = Februa...,NaN,Good Afternoon I hope this email finds you all...,1,NaN,1.0,0.0,0,human,attached,1.0,positive,1.0,14.0,7.1,1.0,0.0
930,75290,45409,437044,False,0,FALSE,decantae.co.uk,admin@decantae.co.uk,admin@decantae.co.uk,"{Invoicepayments.UK@dorchestercollection.com,A...",2,{},{},{},2022-02-09 14:20:09+00,0,NaN,False,NaN,True,True,True,False,False,True,False,False,False,Decantae Mineral Water Ltd Inv No:,Invoice 101609.pdf,Hi The attached credit relates to invoice 9823...,1,NaN,1.0,0.0,0,human,attached,1.0,positive,2.0,7.0,28.6,1.0,0.0
341,6869,6170,223678,False,0,FALSE,artsystems.co.uk,accounts@artsystems.co.uk,accounts@artsystems.co.uk,{purchase.ledger@evo-group.co.uk},1,{},{},{},2021-12-17 16:45:46+00,0,NaN,False,NaN,True,True,True,False,True,False,False,True,False,ArtSystems Month End Statement,Statement CU-10000037.pdf,Attention: This email originated outside of o...,0,NaN,0.0,1.0,0,automatic-reply,0,1.0,negative,0.0,21.0,0.0,0.0,0.0
652,22263,13687,455322,True,10,455322_t,dorchestercollection.com,['pwatts@kendraenergy.co.uk'],Paula Watts,"['Jyoti.Hirani@dorchestercollection.com', 'AP....",3,NaN,NaN,{},15/04/2020 08:04,0,NaN,False,NaN,False,True,True,True,True,False,False,False,False,The Dorchester- Outstanding ¬£4104.00,NaN,"Good Morning, I have not received an update...",1,1.0,NaN,NaN,0,NaN,NaN,NaN,positive,1.0,16.0,6.2,1.0,0.0
685,32678,18996,460631,True,1,460631_t,dorchestercollection.com,['aurelija@balloonista.co.uk'],Aurelija Meakin,['APInvoicequeries.UK@dorchestercollection.com'],1,NaN,NaN,{},29/11/2021 10:18,0,NaN,False,NaN,True,True,True,False,False,False,False,False,False,Balloonista Ltd address and Bank account change,NaN,"Good morning, As discussed earlier, please fi...",1,1.0,NaN,NaN,0,NaN,NaN,NaN,positive,1.0,8.0,12.5,1.0,0.0
598,10984,9923,227430,False,0,FALSE,aquaid-wilts.co.uk,Alex@aquaid-wilts.co.uk,Alex@aquaid-wilts.co.uk,"{Purchase.Ledger@evo-group.co.

In [ ]:
# function for this loop
def sentence_by_sentence(df, model):
    '''Classify the text of the emails in a sentence-by-sentence manner
    '''
    
    email_iter = 1

    sentence_cols = ['this_email','email_id','is_part_of_thread','num_in_thread','thread_id','initial_domain',
                     'from_address','sentence','true_label','true_class','y_pred']
    
    meta_app_cols = ['this_email','true_label','sum_score','email_length','perc_flagged','y_pred','diff']
    
    df_list = []
    meta_df = []

    split_sentence_string = "[.?!]"
    
    # change name of df here
    for email in df[0:10].itertuples():

    #     sentences = email.text.split('.')
        clean_text = clean_websites(email.text)

        sentences = re.split(split_sentence_string,clean_text)
        email_iter += 1
        sum_score = 0

        len_email = len(sentences)

        for s in sentences:

            # check if this bit of the list is empty
            if len(s.strip()) == 0:
                continue
            else:

                preds = model([s])
                this_label = preds.item()
                sum_score += this_label

                row_data = [email_iter, email.email_id, email.is_part_of_thread, email.num_in_thread, email.thread_id,
                    email.initial_domain, email.from_address,
                    s,email.label,email.label_text,this_label]
                
                df_row = pd.DataFrame([row_data],columns = sentence_cols)
                df_list.append(df_row)

        # save when this is completed
        email_df = pd.concat(df_list, ignore_index=True)
                
        # save the email specific info
        perc_flag = "{:.1f}".format((sum_score*100)/len_email)
        
        meta_row = pd.DataFrame(email)#pd.Series(email)#.to_frame()
        print(meta_row)

        meta_row['sum_score'] = sum_score
        meta_row['len_email'] = len_email
        meta_row['perc_flag'] = perc_flag
        
#         meta_df = pd.concat(meta_row)

        meta_df.append(meta_row)
        
#     meta_df = pd.DataFrame(meta_df)

    meta_df = pd.concat(meta_df)
    
    return email_df, meta_df

In [ ]:
%%time
email_df, meta_df = sentence_by_sentence(this_df, model)

In [115]:
meta_df

0                                                                                                                                                                                                                                                                                                                    403
1                                                                                                                                                                                                                                                                                                                  29955
2                                                                                                                                                                                                                                                                                                                  19696
3                                                            

## Quick summary:

Takes 4 min 35 seconds to run over 991 emails

In [30]:
# pandas options
pd.set_option('display.max_rows', 3000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_colwidth', 300)

In [31]:
email_df[0:300]

,this_email,sentence,true_label,true_class,pred_label
0,2,Attention: This email originated outside of our organisation,0,negative,0
1,2,Please be extra vigilant when opening attachments or clicking links,0,negative,0
2,2,For the attention of Accounts Payable Department Please find attached your invoice 00324904 If you have any queries regarding the attached invoice please email Credit Control at,0,negative,0
3,2,uk Yours sincerely Encore Credit Control Team FRAUD WARNING WE WILL NOT CHANGE OUR BANK DETAILS,0,negative,0
4,2,For security reasons and to avoid fraud we strongly recommend that before any payment is sent to this firm that you telephone our office to obtain confirmation that the bank details that you have received are correct,0,negative,0
5,2,"We accept no responsibility or liability for malicious or fraudulent emails purportedly coming from our firm and it is your responsibility to satisfy yourself that emails shown as coming from our firm, are genuine, before relying on anything contained within them",0,negative,0
6,3,"Greetings from Amazon Web Services, We're writing to provide you with an electronic invoice for your use of AWS services",0,negative,0
7,3,"Additional information regarding your bill, individual charge service details, and your account history are available on the Billing & Cost Management Page",0,negative,0
8,3,Please remit payment stating ACCOUNT NUMBER and INVOICE NUMBER the payment is for,0,negative,0
9,3,"Account ID Invoice No Statement Date Payment Due Date 565104697810 995513305 04/02/2022 05/02/2022 Bill To Scripps Media, Inc",0,negative,0


In [31]:
print(len(meta_df))
meta_df["y_pred"] = meta_df.apply(lambda row: 1 if row["sum_score"] > 0 else 0, axis = 1)
meta_df["diffs"] = meta_df.apply(lambda row: 0 if row["y_pred"] == row["true_label"] else 1, axis = 1)
meta_df

996


,this_email,true_label,sum_score,email_length,perc_flagged,y_pred,diffs
0,2,0,1,3,33.3,1,1
1,3,0,0,9,0.0,0,0
2,4,0,0,3,0.0,0,0
3,5,0,0,5,0.0,0,0
4,6,0,0,5,0.0,0,0
5,7,0,0,47,0.0,0,0
6,8,0,0,30,0.0,0,0
7,9,0,0,30,0.0,0,0
8,10,0,0,30,0.0,0,0
9,11,0,2,5,40.0,1,1


In [32]:
pos_exp = email_df.query("pred_label == 1").reset_index(drop=True)
print(len(pos_exp))
pos_exp

684


,this_email,sentence,true_label,true_class,pred_label
0,2,Good Morning Please can you confirm Settlement date of our 2 invoices That you have updated our bank details to the new ones supplied (as attached) Look forward to hearing from you Kind regards Natalie Salama-Levy Managing Director T 020 7624 2013 M 07958 310 420 W W www,0,negative,1
1,11,We will be closing the bank account soon that you are sending our ACH payments to,0,negative,1
2,11,Can you please send me a form so I can give you our new bank account information,0,negative,1
3,15,", we are reaching out to suppliers to switch from paper checks to receiving payments via ACH",0,negative,1
4,26,**Please note that we now take Credit/Debit Cards for payment and we have a Direct Debit facility that all of our Clients can sign up to,0,negative,1
5,37,"In case new linked accounts are added, click 'Enable Tax Settings Inheritance' so the tax registration information will automatically be inherited from your Payer account to your Linked accounts",0,negative,1
6,133,"If you have any questions about your invoice, please email our accounts department: Tel:+44(0) 118 925 8250 As of August, Bottomline Technologies will be updating some of our email addresses and as such the email address from which you receive your invoices and statements from us will change",0,negative,1
7,133,Kindly update your systems/records with our new email address details: no-reply-,0,negative,1
8,134,"If you have any questions about your invoice, please email our accounts department: Tel:+44(0) 118 925 8250 As of August, Bottomline Technologies will be updating some of our email addresses and as such the email address from which you receive your invoices and statements from us will change",0,negative,1
9,134,Kindly update your systems/records with our new email address details: no-reply-,0,negative,1


In [8]:
neg_exp = email_df.head(200).query("pred_label == 0").reset_index(drop=True)
print(len(neg_exp))
neg_exp

NameError: name 'email_df' is not defined

## How many are different?

Here I am filtering on the emails that had recieved a different label from the manual labelling (e.g. the entire email was labelled as a 0 or a 1) and whether the fine-tuned SetFit model had flagged any sentences. 

Differences could arise because:

- the email was labelled a 0, but SetFit has flagged at least 1 sentence as a change of bank details email
- the email was labelled as a 1, but SetFit has flagged 0 sentences

In [46]:
# 9% of the 
perc = (sum(meta_df["diffs"])*100)/len(meta_df)
print(f'Number of diff emails: {sum(meta_df["diffs"])}/{len(meta_df)} ({"{:.1f}".format(perc)}%)')

Number of diff emails: 116/996 (11.6%)


In [36]:
meta_df

,this_email,true_label,sum_score,email_length,perc_flagged,y_pred,diffs
0,2,0,1,3,33.3,1,1
1,3,0,0,9,0.0,0,0
2,4,0,0,3,0.0,0,0
3,5,0,0,5,0.0,0,0
4,6,0,0,5,0.0,0,0
5,7,0,0,47,0.0,0,0
6,8,0,0,30,0.0,0,0
7,9,0,0,30,0.0,0,0
8,10,0,0,30,0.0,0,0
9,11,0,2,5,40.0,1,1


In [34]:
# merge the two dataframes based on the 'id' column
diff_ids = meta_df.loc[meta_df['diffs'] == 1, 'this_email']

# # filter the merged dataframe to keep only the rows where 'different' column is 1
exps = email_df[email_df['this_email'].isin(diff_ids)].reset_index(drop=True)

# print the resulting dataframe
exps

,this_email,sentence,true_label,true_class,pred_label
0,2,Good Morning Please can you confirm Settlement date of our 2 invoices That you have updated our bank details to the new ones supplied (as attached) Look forward to hearing from you Kind regards Natalie Salama-Levy Managing Director T 020 7624 2013 M 07958 310 420 W W www,0,negative,1
1,2,1070kitchen,0,negative,0
2,2,"shop On 24 Nov 2021, at 12:03, Accounts Payable Team - UK",0,negative,0
3,11,We will be closing the bank account soon that you are sending our ACH payments to,0,negative,1
4,11,Can you please send me a form so I can give you our new bank account information,0,negative,1
5,11,Thank you,0,negative,0
6,11,Susan Cherwa Atlantic Blueridge Elevator Company 2505 S,0,negative,0
7,11,"Military Highway, Suite C Chesapeake, VA 23320 PH 757-490-8255 FAX 757-494-8839",0,negative,0
8,15,I will need a secure fax where I can send this information,0,negative,0
9,15,I also need to know if the funds will be coming from a US bank account,0,negative,0


Some highlights:

The email abut hacking (REARDON SMITH ARCHITECTS LTD):

Was manually labelled as negative, and e.g. this sentence did not get flagged:

- it looks like their email was hacked; someone was sending fake information about changes to bank details

but this sentence was:

- The bank details are still the same

This one also gets flagged: (was manually labelled as negative)

- Good afternoon Could you confirm your bank details and send us new statement

But it also seems more sensitive, e.g. this one gets flagged: (but was missed manually)

- Dear Customer, Could you please confirm date of payment of our attached invoice to our new bank account at SOCIETE GENERALE as mentioned in our invoice

This one was also missed during manual labelling, but gets picked up by the model:

- We thank you to check & arrange payment by the end of the week to our new bank details at SOCIETE GENERALE as mentioned in our invoice


However, this sentence got flagged as well:

- I understand that you require our full data to set us up on your system as a supplier, so could you please send me your 'New Suppliers Account' form if applicable


This one got flagged as well:

- I am the new assigned credit controller for all of your accounts so from henceforth would request you to directly send me all the communications regarding the payments so that can log that on the system and you do not receive any automated chasers

This one got flagged too:

- If you have made a payment under the "new" bank details, or receive any further changed details please contact us immediately by phone


## Not getting flagged anymore:

- PLEASE NOTE OUR BANK DETAILS FOR OUR STERLING ACCOUNT ARE AS BELOW: ROYAL BANK OF SCOTLAND SORT CODE 16-26-32 ACCOUNT NUMBER 11526700 If you have any questions regarding your statement please contact us using the details below

- Dear Tetchie, I hope you're well May I know when we can receive the payment for inv 23+38 issued by Capitol Italy, see them attached Amounts are in EUR and we should receive the bank transfer on our Italian bank account BANCA INTESA S	

- 

# Maybe embed sentences from the bat?

In [ ]:
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [ ]:
subset = df.loc[:, ['true_positives','email_text_body'
                    ]].sample(n=500,random_state=42).reset_index(drop=True)

subset

In [ ]:
bodies = subset.pop('email_text_body')

In [ ]:
embedding = embedder.encode(bodies)
embedding.shape

To install umap, we need to install umap-learn not umap

so

pip install umap-learn

and if installed umap:

pip uninstall umap

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Dimension reduction and clustering libraries
import umap
import hdbscan
import sklearn.cluster as cluster
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score

In [ ]:
# cluster = KMeans(n_clusters=2)
# cluster_labels = cluster.fit_predict(umap_embedding)
# real_labels = (subset['true_positives'].values)+1

In [ ]:
# plt.scatter(umap_embedding[:,0],umap_embedding[:,1], c=real_labels, cmap='Spectral', 
#             alpha=0.3)
# plt.legend(['negative','positive'])
# plt.show()

# Use UMAP for the embeddings

Plot reduced dimension semantic similarity of the emails

In [ ]:
subset = df.loc[:, ['true_positives','email_text_body']].sample(n=500,random_state=42).reset_index(drop=True)

subset = subset.rename(columns={"email_text_body":"text","true_positives":"label"})
subset["label_text"] = subset.apply(lambda row: "positive" if row["label"] else "negative", axis = 1)

# randomly sample x from dataset
neg_ex = subset.query("label == 0").reset_index(drop=True)
pos_ex = subset.query("label == 1").reset_index(drop=True)

neg_bodies = neg_ex.pop('text')
pos_bodies = pos_ex.pop('text')

In [ ]:
neg_embed = embedder.encode(neg_bodies)
pos_embed = embedder.encode(pos_bodies)

In [ ]:
pos_embed.shape

In [ ]:
# Create two scatter plots, one for positive and one for negative labels
plt.scatter(pos_embed[:,0], pos_embed[:,1], c='g', alpha=0.3, label='positive')
plt.scatter(neg_embed[:,0], neg_embed[:,1], c='r', alpha=0.3, label='negative')

# Add legend for the two colors
plt.legend()
plt.show()

In [ ]:
neg_umap = umap.UMAP(n_neighbors = 5, n_components = 2, random_state=42).fit_transform(neg_embed)
pos_umap = umap.UMAP(n_neighbors = 5, n_components = 2, random_state=42).fit_transform(pos_embed)

In [ ]:
pd.options.plotting.backend = "plotly"


In [ ]:
# Create two scatter plots, one for positive and one for negative labels
plt.scatter(pos_umap[:,0], pos_umap[:,1], c='g', alpha=0.3, label='positive')
plt.scatter(neg_umap[:,0], neg_umap[:,1], c='r', alpha=0.3, label='negative')

# pos_umap.plot(kind='scatter')

# Add legend for the two colors
plt.legend()
plt.show()

In [ ]:
# import hdbscan

In [ ]:
# umap_embedding = umap.UMAP(n_neighbors = 5, n_components = 2, random_state=42).fit_transform(embedding)

# # hdbscan
# clf = hdbscan.HDBSCAN(min_cluster_size=2)
# clf.fit(umap_embedding.astype(float))
# indices = clf.fit_predict(umap_embedding)

# plt.scatter(umap_embedding, indices)
# plt.show()

# Create training dataset in right format

In [ ]:
# split into train and test set
train_data, test_data = train_test_split(this_df, test_size=0.33, random_state=42)

train_ds = datasets.Dataset.from_dict(train_data.to_dict(orient="list"))
test_ds = datasets.Dataset.from_dict(test_data.to_dict(orient="list"))

In [ ]:
print(type(train_ds))
train_ds

In [ ]:
for text in train_ds:
    print(text)

# Try out the setfit model

In [ ]:
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitModel, SetFitTrainer
from sentence_transformers import SentenceTransformer
import huggingface

In [ ]:
# Load a setfit model from the hub
model_name = "sentence-transformers/paraphrase-mpnet-base-v2"

In [ ]:
# # # import cleaned data
# df = pd.read_csv("data/checked_emails.csv")
# df = df.rename(columns={'bank-detail-location': 'bank_detail_loc'})
# df.head()

# # select 
# subset = df.loc[:, ['email_text_body', # text based fields
#                     'true_positives']] # outcome

# subset = subset.rename(columns={"email_text_body":"text","true_positives":"label"})
# subset["label_text"] = subset.apply(lambda row: "positive" if row["label"] else "negative", axis = 1)

# # randomly sample x from dataset
# neg_ex = subset.query("label == 0").sample(n=10,random_state=42).reset_index(drop=True)
# pos_ex = subset.query("label == 1").sample(n=10,random_state=42).reset_index(drop=True)

# # combine datasets and shuffle them
# this_df = (pd.concat([neg_ex, pos_ex])).sample(frac=1)

# print(this_df.head(10))
# del subset

In [ ]:
# split into train and test set
train_data, test_data = train_test_split(this_df, test_size=0.33, random_state=42)

train_ds = datasets.Dataset.from_dict(train_data.to_dict(orient="list"))
test_ds = datasets.Dataset.from_dict(test_data.to_dict(orient="list"))

In [ ]:
# Load a SetFit model from Hub
# the message below just means that we need to train on some of the labeled examples

model = SetFitModel.from_pretrained(model_name)

In [ ]:
# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    loss_class=CosineSimilarityLoss,
#     batch_size=16,
    num_iterations=20, # Number of text pairs to generate for contrastive learning
    num_epochs=1, # Number of epochs to use for contrastive learning
)

In [ ]:
%%time
# after instantiating the trainer, train it
trainer.train()
metrics = trainer.evaluate()
metrics

In [ ]:
trainer.

# Pure Pytorch model

based on: https://github.com/huggingface/setfit/blob/main/notebooks/text-classification.ipynb

In [ ]:
num_classes = len(train_ds.unique("label"))
model = SetFitModel.from_pretrained(model_name, use_differentiable_head=True, head_params={"out_features": num_classes})

In [ ]:
# instantiate the trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    loss_class=CosineSimilarityLoss,
    num_iterations=20
)

In [ ]:
%%time
# Next, we freeze the weights of the final layer and apply contrastive learning:
trainer.freeze()
trainer.train(body_learning_rate=1e-5, num_epochs=1)

In [ ]:
%%time
#Note that here we can specify the learning rate for the model's body - we find that small values in 1e-5 range work well for this step.
#Now that the model body is tuned, we can unfreeze the head and train it:
#Note that a larger learning rate is used when training the head. We recommend using values in the 1e-2 range. 
#Now that the model is trained, we can evaluate it as usual:
trainer.unfreeze(keep_body_frozen=True)
trainer.train(learning_rate=1e-2, num_epochs=50)

In [ ]:
trainer.evaluate()

In [ ]:
trainer.model()

In [ ]:
# # Create trainer
# trainer = SetFitTrainer(
#     model=model,
#     train_dataset=train_ds,
#     eval_dataset=test_ds,
#     loss_class=CosineSimilarityLoss,
#     metric="accuracy",
#     batch_size=16,
#     num_iterations=20, # The number of text pairs to generate for contrastive learning
#     num_epochs=1 # The number of epochs to use for contrastive learning
# )

# # Train and evaluate
# trainer.freeze() # Freeze the head
# # trainer.train() # Train only the body

# # Unfreeze the head and freeze the body -> head-only training
# # trainer.unfreeze(keep_body_frozen=True)
# # or
# # Unfreeze the head and unfreeze the body -> end-to-end training
# trainer.unfreeze(keep_body_frozen=False)

# trainer.train(
#     num_epochs=25, # The number of epochs to train the head or the whole model (body and head)
#     batch_size=16,
#     body_learning_rate=1e-5, # The body's learning rate
#     learning_rate=1e-2, # The head's learning rate
#     l2_weight=0.0, # Weight decay on **both** the body and head. If `None`, will use 0.01.
# )
# metrics = trainer.evaluate()

In [ ]:
# Train and evaluate!
trainer.train()
metrics = trainer.evaluate()

In [ ]:
metrics

In [ ]:
print(metrics.values)

## SpaCy experimentation

In [ ]:
import spacy
import spacy_sentence_bert

In [ ]:
nlp = spacy_sentence_bert.load_model('en_paraphrase_distilroberta_base_v1')

In [ ]:
doc1 = nlp(neg_ex[0])
doc2 = nlp(neg_ex[9])

In [ ]:
[t for t in doc1]

In [ ]:
[t for t in doc2]

In [ ]:
print(doc1.similarity(doc2))

In [ ]:
g = (doc for doc in nlp.pipe(neg_ex))
g

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [ ]:
embedding = model.encode(pos_ex[0])
embedding

## From tutorial:

Setfit page: https://huggingface.co/blog/setfit

In [ ]:
from datasets import load_dataset
from sentence_transformers.losses import CosineSimilarityLoss

In [ ]:
from setfit import SetFitModel, SetFitTrainer

In [ ]:
dataset = load_dataset("SetFit/SentEval-CR")

### Check out the dataset

In [ ]:
print(type(train_ds))
print(dataset)

In [ ]:
dataset.column_names

In [ ]:
# Select N examples per class (8 in this case)
train_ds = dataset["train"].shuffle(seed=42).select(range(8 * 2))
test_ds = dataset["test"]

In [ ]:
train_ds[0:5]

In [ ]:
# Load SetFit model from Hub
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    loss_class=CosineSimilarityLoss,
    batch_size=16,
    num_iterations=20, # Number of text pairs to generate for contrastive learning
    num_epochs=1 # Number of epochs to use for contrastive learning
)

In [ ]:
# Train and evaluate!
trainer.train()
metrics = trainer.evaluate()

In [ ]:
metrics

In [ ]:
# %%time

# model_name = "en_core_web_sm"
# model = spacy.load(model_name)

# def func(doc):
#     pass

# method = ""

# correct = sum(func(doc) for doc in model.pipe(truepos))
# wrong = sum(func(doc) for doc in model.pipe(falspos))
# precision = correct/(correct + wrong)
# recall = correct/len(truepos)
# accuracy = (correct + len(falspos) - wrong)/(len(truepos) + len(falspos))

# f"{precision},{recall},{accuracy},{model_name},{method}"


In [ ]:
# (res_df
# .assign(precision = lambda d: d.precision.astype(float))
# .assign(accuracy = lambda d: d.accuracy.astype(float))
# .assign(recall = lambda d: d.recall.astype(float))
# [['method', 'model', 'precision', 'recall', 'accuracy']])

In [ ]:
# # nlp pipe example
# g = (doc for doc in nlp.pipe(emails) if func(doc))